In [ ]:
import cv2

In [ ]:
import mediapipe as mp

In [ ]:
# Correct imports using top-level aliases
BaseOptions = mp.tasks.BaseOptions
DetectionResult = mp.tasks.components.containers.DetectionResult
ObjectDetector = mp.tasks.vision.ObjectDetector
ObjectDetectorOptions = mp.tasks.vision.ObjectDetectorOptions
VisionRunningMode = mp.tasks.vision.RunningMode

In [ ]:

# ✅ Replace this with the path to your .tflite model
model_path = "efficientdet_lite0-2.tflite"  # or any object detection TFLite model
import time
import os
assert os.path.exists(model_path), f"Model file not found at {model_path}"

In [ ]:
from utils import streaming_utils_Tut


In [ ]:
#streaming_utils_Tut.getStream(0)

cap = streaming_utils_Tut.getStream(0)

In [ ]:
class Person:
    def __init__(self, id, start, end, center, name = "Unknown", confidence = 0.0):
        self.id = id
        self.start = start
        self.end = end
        self.center = center
        self.name = name
        self.confidence = confidence
        

    def add_name(self, name):
        self.name = name

    def add_confidence(self, confidence):
        self.confidence = confidence

    def add_start(self, start):
        self.box = start
    
    def add_end(self, end):
        self.box = end

    def add_center(self, center):
        self.center = center
    
    def get_id(self):
        return self.id
    
    def get_box(self):
        return self.box
    
    def get_name(self):
        return self.name

    def get_confidence(self):
        return self.confidence

    def get_start(self):
        return self.start

    def get_end(self):
        return self.end

    def get_center(self):
        return self.center

In [ ]:
TrackedPeople = []
import threading

frame_lock = threading.Lock()
shared_frame = None
is_processing = False  # ← This will throttle frame submission
exit_flag = False

def detection_callback(result: DetectionResult, output_image, timestamp_ms: int): #detect_async calls this function when finished i.e. we never call this
    print("Callback triggered")
    global shared_frame, is_processing, exit_flag
    # add drawing code here if needed

    with frame_lock: # while in frame lock, only this function can modify the variables inside, any other will wait. we want the main loop to wait on shared_frame
        if shared_frame is None:
            is_processing = False
            return

        index = 0 # our ID
        if result.detections: #not completely sure if necessary, will have to test
            for detection in result.detections:
                for category in detection.categories:
                    if category.category_name == 'person':
                        #global bbox
                        bbox = detection.bounding_box
                        x1 = int(bbox.origin_x)
                        y1 = int(bbox.origin_y)
                        x2 = int(bbox.origin_x + bbox.width)
                        y2 = int(bbox.origin_y + bbox.height)
                        #creating_object()
                        person = Person(index, (x1,y1), (x2, y2), 0.0, category.category_name, category.score)
                        # person ( id, start, end, center, name, confidence score)
                        index += 1
                        
                        TrackedPeople.append(person)
                        

                  
        # end_time = int(time.time() * 1000)
        # elapsed_time = end_time - start_time
        # print("Time", elapsed_time)
        is_processing = False  # ✅ Now allow main loop to send another frame

options = ObjectDetectorOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=detection_callback,
    score_threshold=0.7,
)

detector = ObjectDetector.create_from_options(options)



while True:
    if exit_flag:
        break

    if is_processing: # while is_processing is true, the if-statent will skip the rest of the loop 
        continue

    ret, local_frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    rgb_frame = cv2.cvtColor(local_frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    with frame_lock:
        shared_frame = local_frame.copy() #pauses everything to copy frame, not sure if completely necessary, will have to try out           


    timestamp_ms = int(time.time() * 1000) #needed variable for live_stream

    is_processing = True 
    detector.detect_async(mp_image, timestamp_ms)

    # Create a fresh frame to draw on
    frame_to_show = local_frame.copy()

    # Safely draw bounding box if detection exists

    # print(len(TrackedPeople)) testing my list to see if multiple people show (not necessary)
    with frame_lock: # not sure if necessary to have frame_lock on but, probably is necessary
        if TrackedPeople:
            
            for person in TrackedPeople:
                start = person.get_start()
                end = person.get_end()
                label = f"{person.get_name()}: {person.get_confidence():.2f}"
                cv2.rectangle(frame_to_show, start, end, (0, 255, 0), 2)
                cv2.putText(frame_to_show, label, (start[0], start[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    TrackedPeople = [p for p in TrackedPeople if p.get_id() == 10] #clears list every time (no one has an id of 10) since I don't have a way to categorize individual people
    # Show frame
    cv2.imshow("Live Detection", frame_to_show)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
        exit_flag = True
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
TrackedPeople = []
import threading

frame_lock = threading.Lock()
shared_frame = None
is_processing = False  # ← This will throttle frame submission
exit_flag = False

def detection_callback(result: DetectionResult, output_image, timestamp_ms: int): #detect_async calls this function when finished i.e. we never call this

    # add drawing code here if needed

    # frame locking

    # specifying that we only want people

    # getting our box corrdinates
    # creating person object
    # adding to list
    
    # finished processing image



#options
# essentially my settings

#creating my detector

#main loop
while True:
    # if statements and get frame

    # image conversion

    # processing image

    # Safely draw bounding box if detection exists

    # drawing on iamge and adding labels

    # displaying image

cap.release()
cv2.destroyAllWindows()
